# Product Price Prediction

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec
import emoji
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

In [55]:
df_product_features_data = pd.read_csv('dataset.csv')

In [56]:
display(df_product_features_data)

,Unique Id,Image,Brand,Title,Specifications,Highlights,Price,Rating
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {...",Make sure this fits by entering your model num...,12.99,4.7
1,f04481fec10c4a538634d01e04bb45ed,https://images-na.ssl-images-amazon.com/images...,YSense,5-6 Packs Baby Girl Cute Headband - Turban Hea...,[{}],"Material: Made of elastic synthetic cotton, so...",9.99,5.0
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,[{'Product Dimensions': '3.2 x 1.8 x 3.2 inche...,Make sure this fits by entering your model num...,379.18,3.3
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,Globalstore 12x25 Folding High-Grade Binocular...,[{'Product Dimensions': '3.9 x 3.9 x 2 inches'...,Make sure this fits by entering your model num...,19.97,5.0
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{'Brand Name': 'GBTIGER'}, {'Item Weight': '2...",Make sure this fits by entering your model num...,27.99,3.0
...,...,...,...,...,...,...,...,...
51495,f9b70f1eccb5487a97bd6c91d0ef85e9,https://images-na.ssl-images-amazon.com/images...,Lucky Line Products,"Lucky Line Key Shapes, ICE Cream, House Key Bl...","[{'Part Number': 'B111K'}, {'Item Weight': '0....",Make sure this fits by entering your model num...,6.75,5.0
51496,af0b9b77549244d09d191e8d52ef2837,https://images-na.ssl-images-amazon.com/images...,Wangdd22,Wangdd22 5mm to 8mm Shaft Rigid Motor Wheel Co...,[{}],not_available,8.99,4.0
51497,d7cea6d9587f498fabe854dbe3d9fc83,https://images-na.ssl-images-amazon.com/images...,by Glistco,VR Carbon Fiber Controller Rifle Adapter for H...,"[{'ASIN': 'B07TBBD5J5'}, {'Customer Reviews': ...",Carbon Fiber Rifle Stock\nFeather Light Materi...,84,0.0
51498,eecc3a4284b948a8a8ecc5885f69d2ef,https://images-na.ssl-images-amazon.com/images...,by NIS America,Ar tonelico: Melody of Elemia,"[{'ASIN': 'B000FUG5TK'}, {'Release date': 'Feb...",Slip cover will hold the artbook & game togeth...,not_available,4.3


In [57]:
df_product_features_data = df_product_features_data[df_product_features_data['Specifications'].apply(lambda x: x != '[{}]')]


In [58]:
df_product_features_data = df_product_features_data[df_product_features_data['Price'].apply(lambda x: x != 'not_available')]

In [59]:
df_product_features_data

,Unique Id,Image,Brand,Title,Specifications,Highlights,Price,Rating
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {...",Make sure this fits by entering your model num...,12.99,4.7
2,754577bc25b24641ab1b0aef5e19db73,https://images-na.ssl-images-amazon.com/images...,Wooden Camera,Wooden Camera - PL Mount (Weapon/Scarlet-W/Epi...,[{'Product Dimensions': '3.2 x 1.8 x 3.2 inche...,Make sure this fits by entering your model num...,379.18,3.3
3,b4c5f6d192124d70919946ea754cce56,https://images-na.ssl-images-amazon.com/images...,Global-store,Globalstore 12x25 Folding High-Grade Binocular...,[{'Product Dimensions': '3.9 x 3.9 x 2 inches'...,Make sure this fits by entering your model num...,19.97,5.0
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{'Brand Name': 'GBTIGER'}, {'Item Weight': '2...",Make sure this fits by entering your model num...,27.99,3.0
5,6b9363bbf2d5454490a59cd8c24c850e,https://images-na.ssl-images-amazon.com/images...,Bogen,Bogen GS150D Gold Series Amplifier,[{'Product Dimensions': '16.5 x 3.5 x 13.5 inc...,Make sure this fits by entering your model num...,449,5.0
...,...,...,...,...,...,...,...,...
51493,11addeeb0ae5436eb53af0205966d632,https://images-na.ssl-images-amazon.com/images...,Spintastics,Spintastics Red Diabolo Pro Chinese Yoyo,"[{'Product Dimensions': '15 x 4 x 5 inches'}, ...",Make sure this fits by entering your model num...,18.99,3.6
51494,276c88eab80e4931954bd6b20e96915a,https://images-na.ssl-images-amazon.com/images...,PeleusTech®,"PeleusTech® Memory Card, 128MB High Speed Memo...",[{'Product Dimensions': '0.4 x 0.4 x 0.4 inche...,Make sure this fits by entering your model num...,8.69,3.9
51495,f9b70f1eccb5487a97bd6c91d0ef85e9,https://images-na.ssl-images-amazon.com/images...,Lucky Line Products,"Lucky Line Key Shapes, ICE Cream, House Key Bl...","[{'Part Number': 'B111K'}, {'Item Weight': '0....",Make sure this fits by entering your model num...,6.75,5.0
51497,d7cea6d9587f498fabe854dbe3d9fc83,https://images-na.ssl-images-amazon.com/images...,by Glistco,VR Carbon Fiber Controller Rifle Adapter for H...,"[{'ASIN': 'B07TBBD5J5'}, {'Customer Reviews': ...",Carbon Fiber Rifle Stock\nFeather Light Materi...,84,0.0


In [60]:
df_product_features_data['Specifications'] = df_product_features_data['Specifications'].str.replace("'", '"')

In [61]:
def is_valid_json(my_json):
    try:
        json.loads(my_json)
        return True
    except ValueError as e:
        return False

In [62]:
df_product_features_data = df_product_features_data[df_product_features_data['Specifications'].apply(is_valid_json)]

In [63]:
df_product_features_data

,Unique Id,Image,Brand,Title,Specifications,Highlights,Price,Rating
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{""Brand"": ""RoadPro""}, {""Model"": ""RP-100C""}, {...",Make sure this fits by entering your model num...,12.99,4.7
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{""Brand Name"": ""GBTIGER""}, {""Item Weight"": ""2...",Make sure this fits by entering your model num...,27.99,3.0
7,00eb206d31bc4000bddffe13e700a602,https://images-na.ssl-images-amazon.com/images...,Photo4Less,Canon EOS Rebel T7i DSLR Camera + EF-S 18-55mm...,"[{""Product Dimensions"": ""13 x 8 x 23 inches""},...",Make sure this fits by entering your model num...,999,0.0
8,267124bb67ec43a2a90649c7a81520ab,https://images-na.ssl-images-amazon.com/images...,Berisfords,"Berisfords 10 mm Razzle Ribbon, Forest and Gold","[{""Item Weight"": ""4.2 ounces""}, {""Package Dime...","Beautiful and luxurious, high quality sparkle ...",5.87,0.0
9,a20c33a8953145dbae6d62b664f0a2d3,https://images-na.ssl-images-amazon.com/images...,Art-Strap,Compatible with Fitbit Versa Leather Watch Wri...,"[{""Package Dimensions"": ""6.9 x 2.2 x 0.5 inche...",Compatible with: FITBIT VERSA\nIncludes 2 Quic...,16.95,0.0
...,...,...,...,...,...,...,...,...
51485,69d34331c2c9478a84678c1c255a6baf,https://images-na.ssl-images-amazon.com/images...,ORB,ORB Vertical Console Stand (PS4),"[{""ASIN"": ""B00GH7UA2I""}, {""Release date"": ""Nov...",Store your console securely in its vertical up...,8.33,0.0
51487,08080e37221649b0b940681f8b3f9363,https://images-na.ssl-images-amazon.com/images...,Deep Silver,Cities Skylines (PS4),"[{""ASIN"": ""B07D6X9QLW""}, {""Customer Reviews"": ...",Build the city of your dreams: Plan road netwo...,24.15,0.0
51491,8f6aff7683da49dea23d89a410d0c66f,https://images-na.ssl-images-amazon.com/images...,Nora Lighting,Nora Lighting NLEDR-67127CW Label LED Retrofit...,"[{""Part Number"": ""NLEDR-67127CW""}, {""Item Weig...",Make sure this fits by entering your model num...,122.1,0.0
51495,f9b70f1eccb5487a97bd6c91d0ef85e9,https://images-na.ssl-images-amazon.com/images...,Lucky Line Products,"Lucky Line Key Shapes, ICE Cream, House Key Bl...","[{""Part Number"": ""B111K""}, {""Item Weight"": ""0....",Make sure this fits by entering your model num...,6.75,5.0


In [64]:
def convert_specs(spec_str):
    try:
        # Attempt JSON parsing
        return json.loads(spec_str)
    except (ValueError, TypeError) as e:
        print("Error occurred:", e)
        return []

df_product_features_data['Specifications'] = df_product_features_data['Specifications'].apply(convert_specs)

C:\Users\hemas\AppData\Local\Temp\ipykernel_9748\1672809656.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_features_data['Specifications'] = df_product_features_data['Specifications'].apply(convert_specs)


In [65]:
df_product_features_data

,Unique Id,Image,Brand,Title,Specifications,Highlights,Price,Rating
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {...",Make sure this fits by entering your model num...,12.99,4.7
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{'Brand Name': 'GBTIGER'}, {'Item Weight': '2...",Make sure this fits by entering your model num...,27.99,3.0
7,00eb206d31bc4000bddffe13e700a602,https://images-na.ssl-images-amazon.com/images...,Photo4Less,Canon EOS Rebel T7i DSLR Camera + EF-S 18-55mm...,"[{'Product Dimensions': '13 x 8 x 23 inches'},...",Make sure this fits by entering your model num...,999,0.0
8,267124bb67ec43a2a90649c7a81520ab,https://images-na.ssl-images-amazon.com/images...,Berisfords,"Berisfords 10 mm Razzle Ribbon, Forest and Gold","[{'Item Weight': '4.2 ounces'}, {'Package Dime...","Beautiful and luxurious, high quality sparkle ...",5.87,0.0
9,a20c33a8953145dbae6d62b664f0a2d3,https://images-na.ssl-images-amazon.com/images...,Art-Strap,Compatible with Fitbit Versa Leather Watch Wri...,[{'Package Dimensions': '6.9 x 2.2 x 0.5 inche...,Compatible with: FITBIT VERSA\nIncludes 2 Quic...,16.95,0.0
...,...,...,...,...,...,...,...,...
51485,69d34331c2c9478a84678c1c255a6baf,https://images-na.ssl-images-amazon.com/images...,ORB,ORB Vertical Console Stand (PS4),"[{'ASIN': 'B00GH7UA2I'}, {'Release date': 'Nov...",Store your console securely in its vertical up...,8.33,0.0
51487,08080e37221649b0b940681f8b3f9363,https://images-na.ssl-images-amazon.com/images...,Deep Silver,Cities Skylines (PS4),"[{'ASIN': 'B07D6X9QLW'}, {'Customer Reviews': ...",Build the city of your dreams: Plan road netwo...,24.15,0.0
51491,8f6aff7683da49dea23d89a410d0c66f,https://images-na.ssl-images-amazon.com/images...,Nora Lighting,Nora Lighting NLEDR-67127CW Label LED Retrofit...,"[{'Part Number': 'NLEDR-67127CW'}, {'Item Weig...",Make sure this fits by entering your model num...,122.1,0.0
51495,f9b70f1eccb5487a97bd6c91d0ef85e9,https://images-na.ssl-images-amazon.com/images...,Lucky Line Products,"Lucky Line Key Shapes, ICE Cream, House Key Bl...","[{'Part Number': 'B111K'}, {'Item Weight': '0....",Make sure this fits by entering your model num...,6.75,5.0


In [66]:
sublist_names = ['Brand', 'Model','Brand Name','Customer Reviews','Shipping Weight','Item model number','Model number','Manufacturer Part Number','OEM Part Number','Brand, Seller, or Collection Name','Warranty Description','Part Number','California residents','ASIN','Item Package Quantity','Number Of Pieces','Manufacturer Part Number','Item Dimensions L x W x H','Date first listed on Amazon']

In [67]:
# Combine Specifications and Highlights into a single text column
combined_text = df_product_features_data['Specifications'].apply(lambda x: ' '.join([str(value) for sublist in x for key, value in sublist.items() if key not in sublist_names and value != 'not_available' and value not in ['NA','Yes','No']])) + ' ' + df_product_features_data['Highlights']


In [68]:
sentences_to_remove = ['Make sure this fits by entering your model number.', '(View shipping rates and policies)']

In [69]:
def remove_sentences(text, sentences):
    for sentence in sentences:
        text = re.sub(re.escape(sentence), '', text)
    return text

In [70]:
combined_text_cleaned = [remove_sentences(str(text), sentences_to_remove) for text in combined_text]

In [71]:
tokenized_documents = [word_tokenize(str(text)) for text in combined_text_cleaned]

In [72]:
english_stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
cleaned_data = [[lemmatizer.lemmatize(word.lower()) for word in document if word.lower() not in english_stopwords and word not in list(string.punctuation)] for document in tokenized_documents]

In [73]:
cleaned_sentences = [' '.join(text) for text in cleaned_data]

In [74]:
special_characters = ['!','"','#','$','%','&','_','(',')','*','+','/',':','','<','=','>','@','[','\\',']','^','`','{','|','}','~','\t', '\s','【','】']
# TODO: Then we need to remove the special characters
for i in range(len(cleaned_sentences)):
    for char in special_characters:
        cleaned_sentences[i] = cleaned_sentences[i].replace(char,"")
cleaned_sentences

['3.2 ounce 5 x 5 x 5 inch rg-58au coax cable pl-259 motorola plug 10 amfm antenna coaxial cable black',
 "2.1 pound 11 x 8.2 x 5 inch anti-crease 120 inch 120 inch projector screen 169 screen format diagonal 120 '' offer 87.01 x 48.82 inch viewing area packaged folded professionally lightweight compact one-piece design easy set up come 18 pack adhesive metal hook stronger plastic hook use two long rope hang outdoor watch movie family also use nail double-sided tape install anti-crease material made high quality synthetic fiber anti-crease material much thicker projector screen project sharper clearer image easy clean soap water more applicable environment suitable home theater business meeting exhibition convention presentation etc best portable foldable projector partner convenient carry storage  compact one piece design fold pack gbtiger professional bag traveling enjoy movie time whenever want",
 "13 x 8 x 23 inch 12.8 pound 1 nonstandard battery battery required photo4less top val

In [75]:
def remove_emojis_other_char(text):
    # Remove single quotes, double quotes
    text = re.sub(r"[\'\"]", "", text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove emojis
    text = emoji.demojize(text)
    return text

In [76]:
cleaned_sentences_new = [remove_emojis_other_char(text) for text in cleaned_sentences]

In [77]:
cleaned_sentences_new

['3.2 ounce 5 x 5 x 5 inch rg-58au coax cable pl-259 motorola plug 10 amfm antenna coaxial cable black',
 '2.1 pound 11 x 8.2 x 5 inch anti-crease 120 inch 120 inch projector screen 169 screen format diagonal 120 offer 87.01 x 48.82 inch viewing area packaged folded professionally lightweight compact one-piece design easy set up come 18 pack adhesive metal hook stronger plastic hook use two long rope hang outdoor watch movie family also use nail double-sided tape install anti-crease material made high quality synthetic fiber anti-crease material much thicker projector screen project sharper clearer image easy clean soap water more applicable environment suitable home theater business meeting exhibition convention presentation etc best portable foldable projector partner convenient carry storage compact one piece design fold pack gbtiger professional bag traveling enjoy movie time whenever want',
 '13 x 8 x 23 inch 12.8 pound 1 nonstandard battery battery required photo4less top value c

In [78]:
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(cleaned_sentences)

In [79]:
tfidf_matrix

<18959x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 515706 stored elements in Compressed Sparse Row format>

In [80]:
encoder = OneHotEncoder()
encoded_brands = encoder.fit_transform(df_product_features_data[['Brand']].astype(str))

In [81]:
encoded_brands

<18959x11248 sparse matrix of type '<class 'numpy.float64'>'
	with 18959 stored elements in Compressed Sparse Row format>

In [82]:
def clean_text(text):
    if not isinstance(text, str):
        return ''  # Return empty string for non-string inputs
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text

In [83]:
df_product_features_data['Cleaned_Title'] = df_product_features_data['Title'].apply(lambda x: clean_text(x) if pd.notnull(x) else '')

C:\Users\hemas\AppData\Local\Temp\ipykernel_9748\2079177568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_features_data['Cleaned_Title'] = df_product_features_data['Title'].apply(lambda x: clean_text(x) if pd.notnull(x) else '')


In [84]:
df_product_features_data

,Unique Id,Image,Brand,Title,Specifications,Highlights,Price,Rating,Cleaned_Title
0,0697f1714bf848c4b1cacc39a46a4e8b,https://images-na.ssl-images-amazon.com/images...,RoadPro,RoadPro RP-100C 10' AM/FM Antenna Coaxial Cable,"[{'Brand': 'RoadPro'}, {'Model': 'RP-100C'}, {...",Make sure this fits by entering your model num...,12.99,4.7,roadpro rp100c 10 amfm antenna coaxial cable
4,b1a1b9c3926b4da1ab0040ea81b41e3e,https://images-na.ssl-images-amazon.com/images...,GBTIGER,"120 inch Portable Projection Screen with Bag, ...","[{'Brand Name': 'GBTIGER'}, {'Item Weight': '2...",Make sure this fits by entering your model num...,27.99,3.0,120 inch portable projection screen bag gbtige...
7,00eb206d31bc4000bddffe13e700a602,https://images-na.ssl-images-amazon.com/images...,Photo4Less,Canon EOS Rebel T7i DSLR Camera + EF-S 18-55mm...,"[{'Product Dimensions': '13 x 8 x 23 inches'},...",Make sure this fits by entering your model num...,999,0.0,canon eos rebel t7i dslr camera efs 1855mm stm...
8,267124bb67ec43a2a90649c7a81520ab,https://images-na.ssl-images-amazon.com/images...,Berisfords,"Berisfords 10 mm Razzle Ribbon, Forest and Gold","[{'Item Weight': '4.2 ounces'}, {'Package Dime...","Beautiful and luxurious, high quality sparkle ...",5.87,0.0,berisfords 10 mm razzle ribbon forest gold
9,a20c33a8953145dbae6d62b664f0a2d3,https://images-na.ssl-images-amazon.com/images...,Art-Strap,Compatible with Fitbit Versa Leather Watch Wri...,[{'Package Dimensions': '6.9 x 2.2 x 0.5 inche...,Compatible with: FITBIT VERSA\nIncludes 2 Quic...,16.95,0.0,compatible fitbit versa leather watch wrist ba...
...,...,...,...,...,...,...,...,...,...
51485,69d34331c2c9478a84678c1c255a6baf,https://images-na.ssl-images-amazon.com/images...,ORB,ORB Vertical Console Stand (PS4),"[{'ASIN': 'B00GH7UA2I'}, {'Release date': 'Nov...",Store your console securely in its vertical up...,8.33,0.0,orb vertical console stand ps4
51487,08080e37221649b0b940681f8b3f9363,https://images-na.ssl-images-amazon.com/images...,Deep Silver,Cities Skylines (PS4),"[{'ASIN': 'B07D6X9QLW'}, {'Customer Reviews': ...",Build the city of your dreams: Plan road netwo...,24.15,0.0,city skyline ps4
51491,8f6aff7683da49dea23d89a410d0c66f,https://images-na.ssl-images-amazon.com/images...,Nora Lighting,Nora Lighting NLEDR-67127CW Label LED Retrofit...,"[{'Part Number': 'NLEDR-67127CW'}, {'Item Weig...",Make sure this fits by entering your model num...,122.1,0.0,nora lighting nledr67127cw label led retrofit ...
51495,f9b70f1eccb5487a97bd6c91d0ef85e9,https://images-na.ssl-images-amazon.com/images...,Lucky Line Products,"Lucky Line Key Shapes, ICE Cream, House Key Bl...","[{'Part Number': 'B111K'}, {'Item Weight': '0....",Make sure this fits by entering your model num...,6.75,5.0,lucky line key shape ice cream house key blank...


In [85]:
title_vectorizer = TfidfVectorizer(max_features=1000)
title_tfidf_matrix = title_vectorizer.fit_transform(df_product_features_data['Cleaned_Title'])

In [86]:
title_tfidf_matrix

<18959x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 108243 stored elements in Compressed Sparse Row format>

In [87]:
combined_features = hstack([title_tfidf_matrix, tfidf_matrix, encoded_brands])

In [88]:
combined_features.shape

(18959, 13248)

In [89]:
X = np.concatenate([combined_features.toarray(), df_product_features_data[['Rating']].values], axis=1)
y = df_product_features_data['Price'].values

In [90]:
X.shape

(18959, 13249)

In [91]:
y.shape

(18959,)

In [92]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=100)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X_scaled)

In [93]:
X_train, X_temp, y_train, y_temp = train_test_split(X_pca, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [94]:
y_train = y_train.astype('float64')
y_val = y_val.astype('float64')
y_test = y_test.astype('float64')

In [95]:
# Train the neural network
model = MLPRegressor(hidden_layer_sizes=(100,)*4, activation='relu', solver='adam', max_iter=1000,early_stopping=True, n_iter_no_change=10, tol=1e-4)
model.fit(X_train, y_train)

MLPRegressor(early_stopping=True, hidden_layer_sizes=(100, 100, 100, 100),
             max_iter=1000)

In [96]:
y_val_pred = model.predict(X_val)

# Calculate mean squared error on validation set
mse = mean_squared_error(y_val, y_val_pred)
print("Validation MSE:", mse)

Validation MSE: 36193.372607416226


In [97]:
r_squared_val = r2_score(y_val, y_val_pred)
print("R-squared value:", r_squared_val)

R-squared value: 0.1644544966574415


In [98]:
y_test_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE:", test_mse)

Test MSE: 38161.05510949968


In [99]:
r_squared = r2_score(y_test, y_test_pred)
print("R-squared value:", r_squared)

R-squared value: 0.15283981430245153


In [100]:
model2 = RandomForestRegressor(n_estimators=100, random_state=42,n_jobs=-1)

In [101]:
model2.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [109]:
y_train_pred_rf = model2.predict(X_train)
r_squared_train_rf = r2_score(y_train, y_train_pred_rf)
print("R-squared value:", r_squared_train_rf)

R-squared value: 0.8843555892487002


In [102]:
y_val_pred_rf = model2.predict(X_val)

# Calculate mean squared error on validation set
mse_rf = mean_squared_error(y_val, y_val_pred_rf)
print("Validation MSE:", mse_rf)

Validation MSE: 38548.3725227975


In [103]:
r_squared_val_rf = r2_score(y_val, y_val_pred_rf)
print("R-squared value:", r_squared_val_rf)

R-squared value: 0.11008792488165364


In [104]:
y_test_pred_rf = model2.predict(X_test)
test_mse_rf = mean_squared_error(y_test, y_test_pred_rf)
print("Test MSE:", test_mse_rf)

Test MSE: 41605.362367338836


In [105]:
r_squared_rf= r2_score(y_test, y_test_pred_rf)
print("R-squared value:", r_squared_rf)

R-squared value: 0.07637756849248001


In [112]:
model_3 = LinearRegression()

In [113]:
model_3.fit(X_train, y_train)

LinearRegression()

In [115]:
y_val_pred_lr = model_3.predict(X_val)

# Calculate mean squared error on validation set
mse_lr = mean_squared_error(y_val, y_val_pred_lr)
print("Validation MSE:", mse_lr)

Validation MSE: 38334.562616540454


In [117]:
r2_score(y_val, y_val_pred_lr)

0.11502385355790423

In [116]:
y_test_pred_lr = model_3.predict(X_test)

# Calculate mean squared error on validation set
mse_lr_test = mean_squared_error(y_test, y_test_pred_lr)
print("Validation MSE:", mse_lr_test)

Validation MSE: 39881.51976430607


In [118]:
r2_score(y_test, y_test_pred_lr)

0.11464618594837017

In [119]:
tokenized_words = [word_tokenize(str(text)) for text in cleaned_sentences]

In [122]:
model_word2Vec = Word2Vec(sentences=tokenized_words, vector_size=100, window=5,  min_count=1, workers=4)

In [ ]:
def text_to_embeddings(text):
    tokens = text.split()  # Assuming text is a single string
    embeddings = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)  # Take the mean of word embeddings
    else:
        return np.zeros(word2vec_model.vector_size)  # Return zero vector if no valid embeddings